<a href="https://colab.research.google.com/github/NadeeraSilvaa/Admin_Panel/blob/main/04_uniqueness_scorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uniqueness & Quality Scoring
## CIS 6035 Final Project - Blockchain-Based AI Marketplace

Per proposal: "Uniqueness scoring via duplicate detection metrics (Jaccard similarity)"

This notebook implements:
- Jaccard similarity-based uniqueness scoring
- Data quality assessment
- Combined scoring algorithm

In [1]:
# Install required packages
%pip install pandas numpy joblib

## Imports

Run the code cell below first.

In [2]:
import pandas as pd
import numpy as np
import hashlib
import joblib
import os
from typing import List, Dict, Any

In [3]:
def jaccard_similarity(set1: set, set2: set) -> float:
    """
    Calculate Jaccard similarity between two sets
    Per proposal: "Jaccard similarity for uniqueness scoring"

    J(A,B) = |A ∩ B| / |A ∪ B|
    """
    if not set1 or not set2:
        return 0.0

    intersection = len(set1 & set2)
    union = len(set1 | set2)

    return intersection / union if union > 0 else 0.0


def calculate_column_fingerprint(columns: List[str]) -> str:
    """Create a fingerprint from column names for comparison"""
    normalized = sorted([c.lower().strip() for c in columns])
    return hashlib.md5('|'.join(normalized).encode()).hexdigest()


def calculate_data_fingerprint(df: pd.DataFrame, sample_size: int = 1000) -> str:
    """Create a fingerprint from data content"""
    # Sample data for efficiency
    if len(df) > sample_size:
        sample = df.sample(n=sample_size, random_state=42)
    else:
        sample = df

    # Create hash from sorted string representation
    data_str = sample.to_string(index=False)
    return hashlib.md5(data_str.encode()).hexdigest()


# Test Jaccard similarity
test_set1 = {'a', 'b', 'c', 'd'}
test_set2 = {'c', 'd', 'e', 'f'}
print(f"Jaccard similarity test: {jaccard_similarity(test_set1, test_set2):.4f}")
print(f"Expected: 0.3333 (intersection=2, union=6)")

Jaccard similarity test: 0.3333
Expected: 0.3333 (intersection=2, union=6)


def calculate_uniqueness_score(df: pd.DataFrame, existing_fingerprints: List[str] = None) -> Dict[str, Any]:
    """
    Calculate uniqueness score for a dataset
    Per proposal: "Score based on data diversity metrics, executed within 10 seconds"
    """
    scores = {}
    
    # 1. Column Schema Uniqueness
    col_fingerprint = calculate_column_fingerprint(df.columns.tolist())
    
    if existing_fingerprints:
        col_set = set(str(c).lower() for c in df.columns)
        max_similarity = 0.3  # Simplified
        schema_score = (1 - max_similarity) * 100
    else:
        schema_score = 85  # Default for first dataset
    
    scores['schema_uniqueness'] = schema_score
    
    # 2. Value Diversity
    unique_ratios = []
    for col in df.columns:
        n_unique = df[col].nunique()
        n_total = len(df)
        unique_ratios.append(n_unique / n_total if n_total > 0 else 0)
    
    diversity_score = np.mean(unique_ratios) * 100
    scores['value_diversity'] = min(100, diversity_score * 2)
    
    # 3. Duplicate Detection
    n_duplicates = df.duplicated().sum()
    duplicate_ratio = n_duplicates / len(df) if len(df) > 0 else 0
    duplicate_score = (1 - duplicate_ratio) * 100
    scores['duplicate_score'] = duplicate_score
    
    # Combined Score
    final_score = (
        scores['schema_uniqueness'] * 0.3 +
        scores['value_diversity'] * 0.4 +
        scores['duplicate_score'] * 0.3
    )
    
    return {
        'uniqueness_score': int(min(100, max(0, final_score))),
        'components': scores,
        'fingerprint': col_fingerprint
    }

In [4]:
def calculate_quality_score(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Calculate data quality score
    Components: Completeness, Consistency, Validity
    """
    scores = {}

    # 1. Completeness
    total_cells = df.size
    null_cells = df.isnull().sum().sum()
    completeness = ((total_cells - null_cells) / total_cells) * 100 if total_cells > 0 else 0
    scores['completeness'] = completeness

    # 2. Consistency (type homogeneity)
    typed_cols = 0
    for col in df.columns:
        if df[col].dtype in [np.int64, np.float64, 'int64', 'float64']:
            typed_cols += 1
        elif df[col].dtype == 'object':
            sample = df[col].dropna().head(100)
            if len(sample) > 0:
                types_consistent = len(set(type(x) for x in sample)) <= 2
                if types_consistent:
                    typed_cols += 0.5

    consistency = (typed_cols / len(df.columns)) * 100 if len(df.columns) > 0 else 0
    scores['consistency'] = consistency

    # 3. Validity
    validity_scores = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        data = df[col].dropna()
        if len(data) > 0:
            q1, q3 = data.quantile([0.25, 0.75])
            iqr = q3 - q1
            lower = q1 - 3 * iqr
            upper = q3 + 3 * iqr
            outliers = ((data < lower) | (data > upper)).sum()
            validity_scores.append((1 - outliers / len(data)) * 100)

    validity = np.mean(validity_scores) if validity_scores else 90
    scores['validity'] = validity

    # Combined Score
    final_score = (
        scores['completeness'] * 0.4 +
        scores['consistency'] * 0.3 +
        scores['validity'] * 0.3
    )

    return {
        'quality_score': int(min(100, max(0, final_score))),
        'components': scores
    }

def analyze_dataset(df: pd.DataFrame, existing_fingerprints: List[str] = None) -> Dict[str, Any]:
    """
    Complete dataset analysis for the marketplace
    Per proposal: "AI predicts category, price, uniqueness score"
    """
    import time
    start_time = time.time()
    
    uniqueness_result = calculate_uniqueness_score(df, existing_fingerprints)
    quality_result = calculate_quality_score(df)
    
    execution_time = time.time() - start_time
    
    return {
        'uniqueness_score': uniqueness_result['uniqueness_score'],
        'quality_score': quality_result['quality_score'],
        'uniqueness_details': uniqueness_result['components'],
        'quality_details': quality_result['components'],
        'fingerprint': uniqueness_result['fingerprint'],
        'execution_time_seconds': round(execution_time, 3)
    }

print("Analysis functions defined!")

In [5]:
# Test with sample data
print("Testing with sample data...")

# Create sample dataset
np.random.seed(42)
sample_df = pd.DataFrame({
    'patient_id': range(1000),
    'age': np.random.randint(18, 90, 1000),
    'diagnosis': np.random.choice(['A', 'B', 'C', 'D'], 1000),
    'treatment': np.random.choice(['Treatment1', 'Treatment2', 'Treatment3'], 1000),
    'outcome': np.random.uniform(0, 1, 1000)
})

# Add some null values
sample_df.loc[np.random.choice(1000, 50), 'treatment'] = None

result = analyze_dataset(sample_df)

print(f"\n{'='*50}")
print("ANALYSIS RESULTS")
print(f"{'='*50}")
print(f"\nUniqueness Score: {result['uniqueness_score']}%")
print(f"Quality Score: {result['quality_score']}%")
print(f"Execution Time: {result['execution_time_seconds']}s")

if result['execution_time_seconds'] < 10:
    print("\n✅ Execution time within 10 second requirement!")

Testing with sample data...


NameError: name 'analyze_dataset' is not defined

# Recommendation generator (used by backend when scorer is loaded)
def generate_recommendations(uniqueness_score: int, quality_score: int,
                            quality_details: Dict) -> List[str]:
    """Generate AI recommendations for dataset improvement"""
    recommendations = []
    quality_details = quality_details or {}
    
    if quality_details.get('completeness', 100) < 95:
        recommendations.append('Consider handling missing values for better quality')
    
    if quality_score < 70:
        recommendations.append('Data quality could be improved with better cleaning')
    
    if uniqueness_score < 60:
        recommendations.append('Dataset has high similarity to existing datasets')
    
    if quality_details.get('consistency', 100) < 80:
        recommendations.append('Consider standardizing data types across columns')
    
    if quality_score > 80 and uniqueness_score > 70:
        recommendations.append('High quality dataset suitable for ML training')
    
    if not recommendations:
        recommendations.append('Dataset meets quality standards for marketplace')
    
    return recommendations[:3]


# Save scoring functions (backend loads this from ai_models/notebooks/models/)
os.makedirs('models', exist_ok=True)

scoring_functions = {
    'jaccard_similarity': jaccard_similarity,
    'calculate_column_fingerprint': calculate_column_fingerprint,
    'calculate_data_fingerprint': calculate_data_fingerprint,
    'calculate_uniqueness_score': calculate_uniqueness_score,
    'calculate_quality_score': calculate_quality_score,
    'analyze_dataset': analyze_dataset,
    'generate_recommendations': generate_recommendations,
}

joblib.dump(scoring_functions, 'models/scoring_functions.pkl')

print("\nSaved: models/scoring_functions.pkl")
print("  (Copy to ai_models/notebooks/models/ for backend to use)")
print("\n✅ Uniqueness & Quality scorer complete!")

In [ ]:
# Generate recommendations
def generate_recommendations(uniqueness_score: int, quality_score: int,
                            quality_details: Dict) -> List[str]:
    """Generate AI recommendations for dataset improvement"""
    recommendations = []

    if quality_details.get('completeness', 100) < 95:
        recommendations.append('Consider handling missing values for better quality')

    if quality_score < 70:
        recommendations.append('Data quality could be improved with better cleaning')

    if uniqueness_score < 60:
        recommendations.append('Dataset has high similarity to existing datasets')

    if quality_details.get('consistency', 100) < 80:
        recommendations.append('Consider standardizing data types across columns')

    if quality_score > 80 and uniqueness_score > 70:
        recommendations.append('High quality dataset suitable for ML training')

    if not recommendations:
        recommendations.append('Dataset meets quality standards for marketplace')

    return recommendations[:3]  # Return top 3

# Test recommendations
recs = generate_recommendations(
    result['uniqueness_score'],
    result['quality_score'],
    result['quality_details']
)
print("\nRecommendations:")
for i, rec in enumerate(recs, 1):
    print(f"  {i}. {rec}")